## Video Classification using 3D Convlolution


In [1]:
# importing necessary packages
import os
import sys

import numpy as np
import pandas as pd

from glob import glob
from matplotlib import pyplot as plt

import seaborn as sns

c:\Users\LEGION\anaconda3\envs\rl\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

import torchmetrics
from torchmetrics import functional
from torchmetrics.classification import BinaryAccuracy

import torchvision.transforms as transforms
from torchvision.transforms._transforms_video import CenterCropVideo, NormalizeVideo

from sklearn.metrics import classification_report

import pytorchvideo
from pytorchvideo.data.encoded_video import EncodedVideo

# Augumentation Process
from pytorchvideo.data import (
    make_clip_sampler,
    labeled_video_dataset,
)

from torchvision.transforms import Compose, RandomHorizontalFlip


from torchvision.transforms import (
    Compose,
    RandomHorizontalFlip,
    Resize,
)

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
)

from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from pytorch_lightning import LightningModule, seed_everything, Trainer

c:\Users\LEGION\anaconda3\envs\rl\lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
c:\Users\LEGION\anaconda3\envs\rl\lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [3]:
# reading the video data using glob
non = glob("Dataset/Dummy/Train/NonViolence/*")
vio = glob("Dataset/Dummy/Train/Violence/*")

label = [0] * len(non) + [1] * len(vio)
df = pd.DataFrame(zip(non + vio, label), columns=["file", "label"])

print("non violence video", len(non))
print("violence video", len(vio))

non violence video 50
violence video 50


In [4]:
# displaying the dataframe
display(df)

,file,label
0,Dataset/Dummy/Train/NonViolence\NV_1.mp4,0
1,Dataset/Dummy/Train/NonViolence\NV_10.mp4,0
2,Dataset/Dummy/Train/NonViolence\NV_11.mp4,0
3,Dataset/Dummy/Train/NonViolence\NV_12.mp4,0
4,Dataset/Dummy/Train/NonViolence\NV_13.mp4,0
...,...,...
95,Dataset/Dummy/Train/Violence\V_50.mp4,1
96,Dataset/Dummy/Train/Violence\V_6.mp4,1
97,Dataset/Dummy/Train/Violence\V_7.mp4,1
98,Dataset/Dummy/Train/Violence\V_8.mp4,1


In [5]:
"""
Normalize the frame extracted from the video
"""
class NormalizeVideo(torch.nn.Module):
    def forward(self, x):
        return x / 255.0

In [6]:
class ResizeVideo(torch.nn.Module):
    def __init__(self, output_size):
        super(ResizeVideo, self).__init__()
        self.resize = transforms.Resize(output_size)

    def forward(self, x):
        # Assuming x is a video tensor with shape (batch_size, num_channels, num_frames, height, width)
        batch_size, num_channels, num_frames, height, width = x.size()
        x_reshaped = x.view(-1, num_channels, height, width)  # Reshape to (batch_size*num_frames, num_channels, height, width)
        x_resized = self.resize(x_reshaped)  # Resize frames
        
        return x_resized.view(batch_size, num_channels, num_frames, x_resized.size(2), x_resized.size(3))  # Reshape back to original shape


In [7]:
# instead of mannually preparing the data pytorchvideo provide library to process the video data
video_transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    # Sampling the video, for this case sampling 10 frames per second
                    UniformTemporalSubsample(10),
                    # Normalizing the video
                    NormalizeVideo(),
                    # Normalizing the frames using mean and standard deviation
                    Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                    # Performing RandomShortSideScale
                    RandomShortSideScale(min_size=248, max_size=256),
                    # Perfoming Center Crop
                    CenterCropVideo(112),
                    # Performing RandomHorizontal Flip
                    RandomHorizontalFlip(p=0.5),
                ]
            ),
        )
    ]
)

In [8]:
# Creating a dataloader for loading data, 
# Augumentation is already done when loading frames
train_dataset = labeled_video_dataset(
    "./Dataset/Dummy",
    clip_sampler=make_clip_sampler("random", 2),
    transform=video_transform,
    decode_audio=False,
)

# loader to load data
loader = DataLoader(train_dataset, batch_size=5, num_workers=0, pin_memory=True)

In [9]:
# checking the shape of inputs
batch = next(iter(loader))
batch.keys()

dict_keys(['video', 'video_name', 'video_index', 'clip_index', 'aug_index', 'label'])

In [10]:
print(f"Dimension of the video: {batch['video'].shape}")
print(f"Dimension of the label: {batch['label'].shape}")

Dimension of the video: torch.Size([5, 3, 10, 112, 112])
Dimension of the label: torch.Size([5])


In [11]:
# visualizing the video
import cv2

# For playing videos
def play_video(video_path):
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        cv2.imshow('Video', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Path to your video file
video_path = "Dataset/Dummy/Test/Violence/V_101.mp4"

# play_video(video_path)


In [12]:
batch = next(iter(loader))

print(f"Dimension of the video: {batch['video'].shape}")
print(f"Dimension of the label: {batch['label'].shape}")

Dimension of the video: torch.Size([5, 3, 10, 112, 112])
Dimension of the label: torch.Size([5])


In [13]:
class MaxPool3D(nn.Module):
    def __init__(self, kernel_size: tuple, stride: tuple=None, padding: tuple=(0, 0, 0)):
        """
        This function initializes the parameters for a maxpool layer

        Parameters
        ------------
            kernel_size : tuple
            window height and width for the maxpooling window

            stride : tuple
            the stride of the window. Default value is kernel_size

            padding: int
            implicit zero padding to be added 
        """
        super(MaxPool3D, self).__init__()
        self.kernel_size = kernel_size
        self.stride = kernel_size if stride is None else stride
        self.padding = padding


    def forward(self, x: torch.tensor): 
        """
        This function performs max-pool operation on the input

        Parameters
        ------------
            x : tensor, float32
            Input image to the convolution layer

        Returns
        ------------
            x : tensor, float32
            max-pooled output from the last layer
        """
        # Returning the max pool operation
        return F.max_pool3d(x, kernel_size=self.kernel_size, stride=self.stride, padding=self.padding)

In [14]:
class Reshape(nn.Module):
    """
    Reshape the input to target shape
    """
    def __init__(self, target_shape: tuple):
        super(Reshape, self).__init__()
        self.target_shape = target_shape
    
    def forward(self, inputs: torch.tensor):
        return inputs.reshape((-1, *self.target_shape))

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

INPUT_DIM = (5, 3, 10, 112, 112)

class Video3DCNN(nn.Module):
    def __init__(self, num_output_features=400):
        super(Video3DCNN, self).__init__()

        self.features = nn.Sequential(
            # first hidden layer
            nn.Conv3d(in_channels=3, out_channels=2, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
            MaxPool3D(kernel_size=(1, 2, 2), stride=(1, 2, 2)),

            # Second hidden layer
            nn.Conv3d(in_channels=2, out_channels=4, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
            MaxPool3D(kernel_size=(2, 2, 2), stride=(2, 2, 2)),

            # Third hidden layer
            nn.Conv3d(in_channels=4, out_channels=8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
            nn.Conv3d(in_channels=8, out_channels=8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
            MaxPool3D(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(1, 1, 1)),

            # Fourth hidden layer
            nn.Conv3d(in_channels=8, out_channels=16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
            nn.Conv3d(in_channels=16, out_channels=16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
            MaxPool3D(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(1, 1, 1)),

            # Fifth hidden layer
            nn.Conv3d(in_channels=16, out_channels=16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
            nn.Conv3d(in_channels=16, out_channels=16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
            MaxPool3D(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(1, 1, 1)),
        )

        # Compute the shape after conv layers dynamically
        self._initialize_fc_layers(INPUT_DIM)

        # Fully connected layers with the specified output sizes
        self.fc1 = nn.Linear(in_features=self.fc_input_features, out_features=800)  # Adjusted to match next layer's input
        self.fc2 = nn.Linear(in_features=800, out_features=1000)  # Adjusted to match next layer's input
        self.fc3 = nn.Linear(in_features=1000, out_features=num_output_features)
        self.softmax = nn.Softmax(dim=1)


    def _initialize_fc_layers(self, input_shape):
        """
        Used to calculate the dimension that gets passed to linear layer from the convolution layer
        """
        with torch.no_grad():
            x = torch.rand(input_shape)
            x = self.features(x)
            self.fc_input_features = x.view(x.size(0), -1).size(1)


    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5)
        x = self.fc3(x)

        return x


# # Create a model instance
# model = Video3DCNN(num_output_features=400)

# # Print model summary
# print(model)

# # Dummy input: Batch of 5 videos, 3 color channels, 10 frames, 112x112 pixels
# dummy_input = torch.randn(5, 3, 10, 112, 112)  # Adjusted the batch size to match the target data

# # Forward pass
# output = model(dummy_input)
# print(output.shape)  # Expected output shape: (5, 400)

# # Sample target for testing loss calculation
# sample_target = torch.randint(0, 400, (5,))  # Assuming 5 samples and 400 classes, batch size matches input data

# # Define a loss function
# criterion = nn.CrossEntropyLoss()
# loss = criterion(output, sample_target)
# print(f"Loss: {loss.item()}")

In [16]:
class VideoClassifier(LightningModule):
    def __init__(self):
        super(VideoClassifier, self).__init__()

        # For storing the hyperparameters
        self.save_hyperparameters()

        # For logs
        self.training_step_loss = []
        self.training_step_metric = []
        self.validation_step_loss = []
        self.validation_step_metric = []

        self.validation_step_y_hats = []
        self.validation_step_ys = []
        
        # Architecture
        self.video_module = Video3DCNN()
        self.relu = nn.ReLU()
        self.linear = nn.Linear(400, 1)

        # Hyperparameters
        self.lr = 1e-3
        self.batch_size = 5
        self.numworker = 3

        # evaluation metric
        self.metric = BinaryAccuracy()

        # loss function
        self.criterion = nn.BCEWithLogitsLoss()

    
    def forward(self, x):
        x = self.video_module(x)
        x = self.relu(x)
        x = self.linear(x)

        return x.squeeze(dim=1)  # Squeeze to make predictions have shape (batch_size,)

In [17]:
def configure_optimizer(self):
    """
    Configuration of optizer and decaying learning rate for the neural network
    """
    opt = torch.optim.AdamW(params=self.parameters(), lr=self.lr)
    scheduler = CosineAnnealingLR(opt, T_max=10, eta_min=1e-6, last_epoch=-1)
    return {"optimizer": opt, "lr_scheduler": scheduler}

VideoClassifier.configure_optimizers = configure_optimizer

In [18]:
def data_loader(path: str):
    """
    Defining the data loader for the train, test and validation set
    """
    dataset = labeled_video_dataset(
        path,
        clip_sampler=make_clip_sampler("random", 1),
        transform=video_transform,
        decode_audio=False,
    )

    loader = DataLoader(dataset, batch_size=5, num_workers=0, pin_memory=True)

    return loader

In [19]:
def train_dataloader(self):
    """
    Train dataloader
    """
    return data_loader(path="Dataset/Dummy/Train")

VideoClassifier.train_dataloader = train_dataloader

In [20]:
def training_step(self, batch, batch_idx):
    """
    Training step for the lightning module
    """
    # Extracting the data and target
    video, label = batch["video"], batch["label"]
    
    # Forward Pass
    out = self(video)

    # Calculation of loss and metric
    loss = self.criterion(out.squeeze(), label.float())
    metric = self.metric(out, label.to(torch.int64))

    # Logging the loss
    self.log("train_loss", loss)

    self.training_step_loss.append(loss)
    self.training_step_metric.append(metric)

    return {"loss": loss, "metric": metric.detach()}

VideoClassifier.training_step = training_step

In [21]:
def on_train_epoch_end(self, *args, **kwargs):
    """
    Whenever the epoch ends, this function runs.
    """
    epoch_avg_loss = torch.stack(self.training_step_loss).mean().detach().cpu().numpy()
    epoch_avg_metric = torch.stack(self.training_step_metric).mean().detach().cpu().numpy()

    self.log("Training Loss", torch.from_numpy(epoch_avg_loss).type(torch.float32))
    self.log("Training Metric", torch.from_numpy(epoch_avg_metric).type(torch.float32))

    # self.training_step_loss.clear()
    # self.training_step_metric.clear()

VideoClassifier.on_train_epoch_end = on_train_epoch_end

In [22]:
def val_dataloader(self):
    """
    loader for validation set.
    """
    return data_loader(path="Dataset/Dummy/Validation")

VideoClassifier.val_dataloader = val_dataloader

In [23]:
def validation_step(self, batch, batch_idx):
    """
    Validation steps
    """
    # Extractino of data and target
    video, label = batch["video"], batch["label"]
    # Forward pass
    out = self(video)
    # Calculation of loss and metric
    loss = self.criterion(out.squeeze(), label.float())
    metric = self.metric(out, label.to(torch.int64))

    # logging the loss
    self.log("val_loss", loss)

    self.validation_step_loss.append(loss)
    self.validation_step_metric.append(metric)

    return {"val_loss": loss, "metric": metric.detach()}

VideoClassifier.validation_step = validation_step

In [24]:
def on_validation_epoch_end(self, *args, **kwargs):
    """
    Whenever the validation ends this function runs.
    """
    epoch_avg_loss = torch.stack(self.validation_step_loss).mean()
    epoch_avg_metric = torch.stack(self.validation_step_metric).mean()

    self.log("Average Validation Loss", epoch_avg_loss)
    self.log("Average Validation Accuracy", epoch_avg_metric)

    self.validation_step_loss.clear()
    self.validation_step_metric.clear()

VideoClassifier.on_validation_epoch_end = on_validation_epoch_end

In [25]:
def test_dataloader(self):
    """
    loader for test set.
    """
    return data_loader(path="Dataset/Dummy/Test")

VideoClassifier.test_dataloader = test_dataloader

In [26]:
@torch.no_grad()
def test_step(self, batch, batch_idx):
    """
    Testing step
    """
    # Extracting the data and label
    video, label = batch["video"], batch["label"]

    # Forward pass
    pred = self(video)

    # Calculation of loss
    loss = self.criterion(pred.squeeze(), label.float())

    self.log("test_loss", loss)
    self.log("length pred", len(pred))
    
    treshold = 0.05

    # Transfering to CPU 
    pred = pred.detach().cpu()
    label = label.detach().cpu()
    
    # Calculation of accuracy, f1_score, precision, recall
    accuracy = functional.accuracy(pred, label, task="binary")
    f1_score_pred = functional.f1_score(pred, label, task="binary", average="weighted", threshold=treshold)
    confmat = functional.confusion_matrix(pred, label, task="binary")
    precision = functional.precision(pred, label, task="binary", threshold=treshold)
    recall = functional.recall(pred, label, task="binary", threshold=treshold)

    self.log("Precision",precision)
    self.log("Recall",recall)
    self.log("test_accuracy", accuracy)
    self.log("train_f1", f1_score_pred)
    
    self.validation_step_y_hats.append(pred)
    self.validation_step_ys.append(label)
        
    return {"preds": pred, "targets": label}

VideoClassifier.test_step = test_step

In [27]:
@torch.no_grad()
def on_test_epoch_end(self):
    """
    Whenever the test is performed, this function runs.
    """
    pred = torch.cat(self.validation_step_y_hats)
    label = torch.cat(self.validation_step_ys)

    confusion_matrix = torchmetrics.ConfusionMatrix(task="binary", num_classes=2, threshold=0.05)
    confusion_matrix(pred, label.int())

    confusion_matrix_computed = confusion_matrix.compute().detach().cpu().numpy().astype(int)

    df_cm = pd.DataFrame(confusion_matrix_computed)
    plt.figure(figsize = (10, 7))
    fig_ = sns.heatmap(df_cm, annot=True, cmap="Spectral").get_figure()
    plt.close(fig_)
    
    self.loggers[0].experiment.add_figure("Confusion matrix", fig_, self.current_epoch)

VideoClassifier.on_test_epoch_end = on_test_epoch_end

In [28]:
# Model Checkpointing
checkpoint_callback = ModelCheckpoint(monitor="val_loss", dirpath="checkpoints",
                                      filename="file", save_last=True)
lr_monitor = LearningRateMonitor(logging_interval="epoch")

In [29]:
# Creating and training the model

# Initializing the model
model = VideoClassifier()

# seeding for reproducibility
seed_everything(1)

# Initializing the trainer
trainer = Trainer(min_epochs=10,
                  max_epochs=15,
                  accelerator="gpu" if torch.cuda.is_available() else "cpu",
                  devices=-1 if torch.cuda.is_available() else 1,
                  precision="16-mixed",
                  accumulate_grad_batches=2,
                  enable_checkpointing=True,
                  enable_progress_bar=True,
                  enable_model_summary=True,
                  num_sanity_val_steps=0,
                  callbacks=[lr_monitor, checkpoint_callback],
                  limit_predict_batches=5,
                  limit_val_batches=5,
                  limit_test_batches=5
                )

Seed set to 1
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [30]:
# Training the model
trainer.fit(model)

You are using a CUDA device ('NVIDIA GeForce RTX 3050 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: c:\Users\LEGION\Documents\My Documents\KU\First Semester\Data Analytics\Video Classification using 3D Convolution\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type              | Params
---------------------------------------------------
0 | video_module | Video3DCNN        | 1.9 M 
1 | relu         | ReLU              | 0     
2 | linear       | Linear            | 401   
3 | metric       | BinaryAccuracy    | 0     
4 | criterion    | BCEWithLogitsLoss | 0     
---------------------------------------------------
1.9 M     Trainable params
0         Non-traina

Epoch 0: |          | 20/? [00:45<00:00,  0.44it/s, v_num=0]

c:\Users\LEGION\anaconda3\envs\rl\lib\site-packages\pytorch_lightning\utilities\data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 5. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Epoch 14: |          | 20/? [00:42<00:00,  0.47it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: |          | 20/? [00:42<00:00,  0.47it/s, v_num=0]


In [31]:
# Checking the validation result
val_res = trainer.validate(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\LEGION\anaconda3\envs\rl\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Validation DataLoader 0:  40%|████      | 2/5 [00:00<00:00,  4.30it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Validate metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Average Validation Accuracy    0.6000000238418579
  Average Validation Loss       1.151508092880249
         val_loss               1.151508092880249
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [32]:
# Checking the test result
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\LEGION\anaconda3\envs\rl\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing DataLoader 0:  40%|████      | 2/5 [00:00<00:01,  2.65it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        Precision                  0.25
         Recall                     0.5
       length pred                  5.0
      test_accuracy         0.4000000059604645
        test_loss           1.6746532917022705
        train_f1            0.3333333432674408
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.6746532917022705,
  'length pred': 5.0,
  'Precision': 0.25,
  'Recall': 0.5,
  'test_accuracy': 0.4000000059604645,
  'train_f1': 0.3333333432674408}]

In [33]:
import pickle

# Saving the model
def save_model_pickle(model, filepath):
    with open(filepath, 'wb') as f:
        pickle.dump(model, f)


# Save the model
save_model_pickle(model, "models/3DCNN_video_classifier_v1.pkl")

In [53]:
from pytorchvideo.data.encoded_video import EncodedVideo

# Predictin the realword case scenario usecase
# play_video("Dataset/Real Life Violence Dataset/Validation/Violence/V_436.mp4")

# video = EncodedVideo.from_path("Dataset/Real Life Violence Dataset/Validation/Violence/V_436.mp4")

In [54]:
play_video("Dataset/Real Life Violence Dataset/Validation/NonViolence/NV_267.mp4")

video = EncodedVideo.from_path("Dataset/Real Life Violence Dataset/Validation/NonViolence/NV_267.mp4")

In [55]:
# Transforming the video
video_data = video.get_clip(0, 2)
video_data = video_transform(video_data)
video_data["video"].shape

torch.Size([3, 10, 112, 112])

In [56]:
# Transferming the model to GPU
model = model.cuda()

# Transfering the input to GPU
inputs = video_data["video"].cuda()

# Adding Dimension as the input must be in form (batch_size, number_of_channel, no_of_frame, width, height)
inputs = torch.unsqueeze(inputs, 0) # adding dimensions
inputs.shape

torch.Size([1, 3, 10, 112, 112])

In [57]:
# Predicting the result
preds = model(inputs)
preds = preds.detach().cpu().numpy()
preds

array([-1.8530151], dtype=float32)

In [58]:
# Probability of the prediction
preds[0]

-1.8530151

In [59]:
# Converting the probability to class
preds = np.where(preds>0.5, 1, 0)
preds[0]

0

In [52]:
# If the VSCode has tensorboard extension, run this to start the tensorboard
# %load_ext tensorboard
# %tensorboard --logdir lightning_logs

_Logs are generated in tensorboard: to run tensorboard run <strong>tensorboard --logdir lightning_logs</strong>_